In [1]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting Started With MCP Toolbox

This guide demonstrates how to quickly run
[Toolbox](https://github.com/googleapis/genai-toolbox) end-to-end in Google
Colab using Python, BigQuery, and either [Google
GenAI](https://pypi.org/project/google-genai/), [ADK](https://google.github.io/adk-docs/),
[Langgraph](https://www.langchain.com/langgraph)
or [LlamaIndex](https://www.llamaindex.ai/).

Within this Colab environment, you'll
- Set up a `BigQuery Dataset`.
- Launch a Toolbox server.
- Connect to Toolbox and develop a sample `Hotel Booking` application.

## Step 1: Set up your dataset

In this section, we will
1. Create a dataset in your bigquery project.
1. Insert example data into the dataset.

In [2]:
# @markdown Please fill in the value below and then run the cell.
BIGQUERY_PROJECT = "" # @param {type:"string"}
DATASET = "toolbox_ds" # @param {type:"string"}
TABLE_ID = "hotels" # @param {type:"string"}

In [3]:
# Create the dataset if it does not exist
from google.cloud import bigquery
from google.colab import auth
from google.cloud import exceptions

# Authenticate the user for Google Cloud access
auth.authenticate_user()

bqclient = bigquery.Client(project=BIGQUERY_PROJECT)
dataset_ref = bqclient.dataset(DATASET)

# Check if the dataset already exists
try:
    bqclient.get_dataset(dataset_ref)
    print(f"Dataset {DATASET} already exists. Skipping creation.")
except exceptions.NotFound:
    # If a google.cloud.exceptions.NotFound error is raised, the dataset does not exist.
    print(f"Dataset {DATASET} not found. Creating dataset...")
    dataset = bigquery.Dataset(dataset_ref)
    dataset = bqclient.create_dataset(dataset)
    print(f"Dataset {DATASET} created successfully.")


Dataset toolbox_ds not found. Creating dataset...
Dataset toolbox_ds created successfully.


In [4]:
table_ref = dataset_ref.table(TABLE_ID)

schema = [
    bigquery.SchemaField("id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("name", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("location", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("price_tier", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("checkin_date", "DATE", mode="REQUIRED"),
    bigquery.SchemaField("checkout_date", "DATE", mode="REQUIRED"),
    bigquery.SchemaField("booked", "BOOLEAN", mode="REQUIRED"),
]

# Check if the table already exists; if not, create it and insert data
try:
    bqclient.get_table(table_ref)
    raise ValueError(f"Table '{TABLE_ID}' already exists in dataset '{DATASET}'. Please delete it or use a different table name.")
except exceptions.NotFound:
    table = bigquery.Table(table_ref, schema=schema)
    table = bqclient.create_table(table)
    print(f"Created table '{TABLE_ID}'.")

    sql = f"""
    INSERT INTO `{BIGQUERY_PROJECT}.{DATASET}.{TABLE_ID}`(id, name, location, price_tier, checkin_date, checkout_date, booked)
    VALUES
      (1, 'Hilton Basel', 'Basel', 'Luxury', '2024-04-22', '2024-04-20', FALSE),
      (2, 'Marriott Zurich', 'Zurich', 'Upscale', '2024-04-14', '2024-04-21', FALSE),
      (3, 'Hyatt Regency Basel', 'Basel', 'Upper Upscale', '2024-04-02', '2024-04-20', FALSE),
      (4, 'Radisson Blu Lucerne', 'Lucerne', 'Midscale', '2024-04-24', '2024-04-05', FALSE),
      (5, 'Best Western Bern', 'Bern', 'Upper Midscale', '2024-04-23', '2024-04-01', FALSE),
      (6, 'InterContinental Geneva', 'Geneva', 'Luxury', '2024-04-23', '2024-04-28', FALSE),
      (7, 'Sheraton Zurich', 'Zurich', 'Upper Upscale', '2024-04-27', '2024-04-02', FALSE),
      (8, 'Holiday Inn Basel', 'Basel', 'Upper Midscale', '2024-04-24', '2024-04-09', FALSE),
      (9, 'Courtyard Zurich', 'Zurich', 'Upscale', '2024-04-03', '2024-04-13', FALSE),
      (10, 'Comfort Inn Bern', 'Bern', 'Midscale', '2024-04-04', '2024-04-16', FALSE);
    """
    job = bqclient.query(sql)
    job.result()
    print("Data inserted into 'hotels' table.")

sql_select = f"SELECT * FROM `{BIGQUERY_PROJECT}.{DATASET}.{TABLE_ID}`"
query_job = bqclient.query(sql_select)

print("\nTable Content:")
query_job.to_dataframe()

Created table 'hotels'.
Data inserted into 'hotels' table.

Table Content:


,id,name,location,price_tier,checkin_date,checkout_date,booked
0,1,Hilton Basel,Basel,Luxury,2024-04-22,2024-04-20,False
1,6,InterContinental Geneva,Geneva,Luxury,2024-04-23,2024-04-28,False
2,10,Comfort Inn Bern,Bern,Midscale,2024-04-04,2024-04-16,False
3,4,Radisson Blu Lucerne,Lucerne,Midscale,2024-04-24,2024-04-05,False
4,5,Best Western Bern,Bern,Upper Midscale,2024-04-23,2024-04-01,False
5,8,Holiday Inn Basel,Basel,Upper Midscale,2024-04-24,2024-04-09,False
6,3,Hyatt Regency Basel,Basel,Upper Upscale,2024-04-02,2024-04-20,False
7,7,Sheraton Zurich,Zurich,Upper Upscale,2024-04-27,2024-04-02,False
8,9,Courtyard Zurich,Zurich,Upscale,2024-04-03,2024-04-13,False
9,2,Marriott Zurich,Zurich,Upscale,2024-04-14,2024-04-21,False


## Step 2: Install and configure Toolbox

In this section, we will
1. Download the latest version of the toolbox binary.
2. Create a toolbox config file.
3. Start a toolbox server using the config file.



Download the [latest](https://github.com/googleapis/genai-toolbox/releases) version of Toolbox as a binary.

In [5]:
! curl -O https://storage.googleapis.com/genai-toolbox/v0.4.0/linux/amd64/toolbox

# Make the binary executable
! chmod +x toolbox

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 78.5M  100 78.5M    0     0  55.9M      0  0:00:01  0:00:01 --:--:-- 55.9M


In [6]:
TOOLBOX_BINARY_PATH = "/content/toolbox"
SERVER_PORT = 5000

Create a tools file with the following functions:

- `Database Connection (sources)`: `Includes details for connecting to our hotels database.`
- `Tool Definitions (tools)`: `Defines five tools for database interaction:`
  - `search-hotels-by-name`
  - `search-hotels-by-location`
  - `book-hotel`
  - `update-hotel`
  - `cancel-hotel`

Our application will leverage these tools to interact with the hotels table.

For detailed configuration options, please refer to the [Toolbox documentation](https://googleapis.github.io/genai-toolbox/getting-started/configure/).



In [7]:
# Create a tools file at runtime.
# You can also upload a tools file and use that to run toolbox.
tools_file_name = "tools.yml"
file_content = f"""
sources:
  my-bigquery-source:
    kind: bigquery
    project: {BIGQUERY_PROJECT}
tools:
  search-hotels-by-name:
    kind: bigquery-sql
    source: my-bigquery-source
    description: Search for hotels based on name.
    parameters:
      - name: name
        type: string
        description: The name of the hotel.
    statement: SELECT * FROM `{DATASET}.{TABLE_ID}` WHERE LOWER(name) LIKE LOWER(CONCAT('%', @name, '%'));
  search-hotels-by-location:
    kind: bigquery-sql
    source: my-bigquery-source
    description: Search for hotels based on location.
    parameters:
      - name: location
        type: string
        description: The location of the hotel.
    statement: SELECT * FROM `{DATASET}.{TABLE_ID}` WHERE LOWER(location) LIKE LOWER(CONCAT('%', @location, '%'));
  book-hotel:
    kind: bigquery-sql
    source: my-bigquery-source
    description: >-
       Book a hotel by its ID. If the hotel is successfully booked, returns a NULL, raises an error if not.
    parameters:
      - name: hotel_id
        type: integer
        description: The ID of the hotel to book.
    statement: UPDATE `{DATASET}.{TABLE_ID}` SET booked = TRUE WHERE id = @hotel_id;
  update-hotel:
    kind: bigquery-sql
    source: my-bigquery-source
    description: >-
      Update a hotel's check-in and check-out dates by its ID. Returns a message indicating whether the hotel was successfully updated or not.
    parameters:
      - name: checkin_date
        type: string
        description: The new check-in date of the hotel.
      - name: checkout_date
        type: string
        description: The new check-out date of the hotel.
      - name: hotel_id
        type: integer
        description: The ID of the hotel to update.
    statement: >-
      UPDATE `{DATASET}.{TABLE_ID}` SET checkin_date = PARSE_DATE('%Y-%m-%d', @checkin_date), checkout_date = PARSE_DATE('%Y-%m-%d', @checkout_date) WHERE id = @hotel_id;
  cancel-hotel:
    kind: bigquery-sql
    source: my-bigquery-source
    description: Cancel a hotel by its ID.
    parameters:
      - name: hotel_id
        type: integer
        description: The ID of the hotel to cancel.
    statement: UPDATE `{DATASET}.{TABLE_ID}` SET booked = FALSE WHERE id = @hotel_id;
toolsets:
  my-toolset:
    - search-hotels-by-name
    - search-hotels-by-location
    - book-hotel
    - update-hotel
    - cancel-hotel
"""

In [8]:
with open(tools_file_name, 'w', encoding='utf-8') as f:
    f.write(file_content)

In [9]:
TOOLS_FILE_PATH = f"/content/{tools_file_name}"

In [10]:
# Start a toolbox server
! nohup {TOOLBOX_BINARY_PATH} --tools_file {TOOLS_FILE_PATH} -p {SERVER_PORT} > toolbox.log 2>&1 &

In [11]:
# Check if toolbox is running
!sudo lsof -i :{SERVER_PORT}

COMMAND   PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
toolbox 29938 root    7u  IPv4 738791      0t0  TCP localhost:5000 (LISTEN)


## Step 3: Connect your agent to Toolbox

In this section, you will
1. Establish a connection to the tools by creating a Toolbox client.
2. Build an agent that leverages the tools and an LLM for Hotel Booking functionality.


> You need to authenticate as an IAM user so this notebook can access your Google Cloud Project. This access is necessary to use Google's LLM models.

In [12]:
# Run this and allow access through the pop-up
from google.colab import auth

auth.authenticate_user()

In [13]:
# @markdown Please fill in the value below with your GCP project ID and then run the cell.

# Please fill in these values.
project_id = "bigframes-dev-perf"  # @param {type:"string"}

# Quick input validations.
assert project_id, "⚠️ Please provide a Google Cloud project ID"

# Configure gcloud.
!gcloud config set project {project_id}

Updated property [core/project].


> You can either use LangGraph or LlamaIndex to develop a Toolbox based application. Run one of the [Connect Using LangGraph](#scrollTo=pbapNMhhL33S) or [Connect using LlamaIndex](#scrollTo=04iysrm_L_7v&line=1&uniqifier=1) sections below.



### Connect Using LangGraph

In [14]:
# Install the Toolbox Langchain package
!pip install toolbox-langchain --quiet
!pip install langgraph --quiet

# Install the Langchain llm package
# TODO(developer): replace this with another model if needed
! pip install langchain-google-vertexai --quiet
# ! pip install langchain-google-genai
# ! pip install langchain-anthropic

Create a LangGraph Hotel Agent which can Search, Book and Cancel hotels.

In [15]:
from langgraph.prebuilt import create_react_agent
# TODO(developer): replace this with another import if needed
from langchain_google_vertexai import ChatVertexAI
# from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_anthropic import ChatAnthropic
from langgraph.checkpoint.memory import MemorySaver

from toolbox_langchain import ToolboxClient
import pprint
prompt = """
  You're a helpful hotel assistant. You handle hotel searching, booking and
  cancellations. When the user searches for a hotel, mention it's name, id,
  location and price tier. Always mention hotel id while performing any
  searches. This is very important for any operations. For any bookings or
  cancellations, please provide the appropriate confirmation. Be sure to
  update checkin or checkout dates if mentioned by the user.
  Don't ask for confirmations from the user.
"""

queries = [
    "Find hotels in Basel with Basel in it's name.",
    "Can you book the Hilton Basel for me?",
    "Oh wait, this is too expensive. Please cancel it and book the Hyatt Regency instead.",
    "My check in dates would be from April 10, 2024 to April 19, 2024.",
]

# Create an LLM to bind with the agent.
# TODO(developer): replace this with another model if needed
model = ChatVertexAI(model_name="gemini-1.5-pro", project=project_id)
# model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
# model = ChatAnthropic(model="claude-3-5-sonnet-20240620")

# Load the tools from the Toolbox server
client = ToolboxClient("http://127.0.0.1:5000")
tools = client.load_toolset()

# Create a Langraph agent
agent = create_react_agent(model, tools, checkpointer=MemorySaver())
config = {"configurable": {"thread_id": "thread-1"}}
for query in queries:
    inputs = {"messages": [("user", prompt + query)]}
    response = agent.invoke(inputs, stream_mode="values", config=config)
    print(response["messages"][-1].content)

I found the following hotels in Basel:

* **Hilton Basel**, ID: 1, Location: Basel, Price Tier: Luxury
* **Holiday Inn Basel**, ID: 8, Location: Basel, Price Tier: Upper Midscale
* **Hyatt Regency Basel**, ID: 3, Location: Basel, Price Tier: Upper Upscale

You are booked at the Hilton Basel, ID: 1.

Your booking at Hilton Basel, ID: 1 has been cancelled. You are now booked at Hyatt Regency Basel, ID: 3.


Your check-in and check-out dates for Hyatt Regency Basel, ID: 3 have been updated to April 10, 2024 and April 19, 2024 respectively.



### Connect using LlamaIndex

In [16]:
# Install the Toolbox LlamaIndex package
!pip install toolbox-llamaindex --quiet

# Install the llamaindex llm package
# TODO(developer): replace this with another model if needed
! pip install llama-index-llms-google-genai --quiet
# ! pip install llama-index-llms-anthropic

Create a LlamaIndex Hotel Agent which can Search, Book and Cancel hotels.

In [17]:
import asyncio
import os

from llama_index.core.agent.workflow import AgentWorkflow

from llama_index.core.workflow import Context

# TODO(developer): replace this with another import if needed
from llama_index.llms.google_genai import GoogleGenAI
# from llama_index.llms.anthropic import Anthropic

from toolbox_llamaindex import ToolboxClient

prompt = """
  You're a helpful hotel assistant. You handle hotel searching, booking and
  cancellations. When the user searches for a hotel, mention it's name, id,
  location and price tier. Always mention hotel ids while performing any
  searches. This is very important for any operations. For any bookings or
  cancellations, please provide the appropriate confirmation. Be sure to
  update checkin or checkout dates if mentioned by the user.
  Don't ask for confirmations from the user.
"""

queries = [
    "Find hotels in Basel with Basel in it's name.",
    "Can you book the Hilton Basel for me?",
    "Oh wait, this is too expensive. Please cancel it and book the Hyatt Regency instead.",
    "My check in dates would be from April 10, 2024 to April 19, 2024.",
]

async def run_agent():
    # Create an LLM to bind with the agent.
    # TODO(developer): replace this with another model if needed
    llm = GoogleGenAI(
        model="gemini-2.0-flash",
        vertexai_config={"project": project_id, "location": "us-central1"},
    )
    # llm = GoogleGenAI(
    #     api_key=os.getenv("GOOGLE_API_KEY"),
    #     model="gemini-1.5-pro",
    # )
    # llm = Anthropic(
    #   model="claude-3-7-sonnet-latest",
    #   api_key=os.getenv("ANTHROPIC_API_KEY")
    # )

    # Load the tools from the Toolbox server
    client = ToolboxClient("http://127.0.0.1:5000")
    tools = client.load_toolset()

    # Create a LlamaIndex agent
    agent = AgentWorkflow.from_tools_or_functions(
        tools,
        llm=llm,
        system_prompt=prompt,
    )

    # Run the agent
    ctx = Context(agent)
    for query in queries:
        response = await agent.run(user_msg=query, ctx=ctx)
        print(f"---- {query} ----")
        print(str(response))

await run_agent()

---- Find hotels in Basel with Basel in it's name. ----
I found the following hotels in Basel with Basel in their name: Hilton Basel (ID: 1) located in Basel with price tier Luxury, Holiday Inn Basel (ID: 8) located in Basel with price tier Upper Midscale and Hyatt Regency Basel (ID: 3) located in Basel with price tier Upper Upscale.
---- Can you book the Hilton Basel for me? ----
OK. I have booked the Hilton Basel for you.

---- Oh wait, this is too expensive. Please cancel it and book the Hyatt Regency instead. ----
OK. I have cancelled the Hilton Basel and booked the Hyatt Regency for you.

---- My check in dates would be from April 10, 2024 to April 19, 2024. ----
OK. I have updated your check-in and check-out dates for Hyatt Regency Basel.



### Connect Using ADK

In [18]:
!pip install -q google-adk

In [19]:
# Create an ADK Hotel Agent which can Search, Book and Cancel hotels.
from google.adk.agents import Agent
from google.adk.tools.toolbox_tool import ToolboxTool
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.artifacts.in_memory_artifact_service import InMemoryArtifactService
from google.genai import types

import os

os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'
os.environ['GOOGLE_CLOUD_PROJECT'] = 'bigframes-dev'
os.environ['GOOGLE_CLOUD_LOCATION'] = 'us-central1'

toolbox_tools = ToolboxTool("http://127.0.0.1:5000")

prompt = """
  You're a helpful hotel assistant. You handle hotel searching, booking and
  cancellations. When the user searches for a hotel, mention it's name, id,
  location and price tier. Always mention hotel ids while performing any
  searches. This is very important for any operations. For any bookings or
  cancellations, please provide the appropriate confirmation. Be sure to
  update checkin or checkout dates if mentioned by the user.
  Don't ask for confirmations from the user.
"""

root_agent = Agent(
    model='gemini-2.0-flash',
    name='hotel_agent',
    description='A helpful AI assistant.',
    instruction=prompt,
    tools=toolbox_tools.get_toolset("my-toolset"),
)

session_service = InMemorySessionService()
artifacts_service = InMemoryArtifactService()
session = session_service.create_session(
    state={}, app_name='hotel_agent', user_id='123'
)
runner = Runner(
    app_name='hotel_agent',
    agent=root_agent,
    artifact_service=artifacts_service,
    session_service=session_service,
)

queries = [
    "Find hotels in Basel with Basel in it's name.",
    "Can you book the Hilton Basel for me?",
    "Oh wait, this is too expensive. Please cancel it and book the Hyatt Regency instead.",
    "My check in dates would be from April 10, 2024 to April 19, 2024.",
]

for query in queries:
    content = types.Content(role='user', parts=[types.Part(text=query)])
    events = runner.run(session_id=session.id,
                        user_id='123', new_message=content)

    responses = (
      part.text
      for event in events
      for part in event.content.parts
      if part.text is not None
    )

    for text in responses:
      print(text)

I found three hotels in Basel that have Basel in their name: Hyatt Regency Basel (ID: 3), Holiday Inn Basel (ID: 8), and Hilton Basel (ID: 1).


OK. I have booked the Hilton Basel for you.


OK. I have cancelled the Hilton Basel booking and booked the Hyatt Regency for you instead.
It looks like you're already booked into the Hyatt Regency Basel from April 10 to April 19. Is there anything else?



### Observe the output

You can see that the `Hyatt Regency Basel` has been booked for the correct dates.

In [20]:
sql_select = f"SELECT * FROM `{BIGQUERY_PROJECT}.{DATASET}.{TABLE_ID}`"
query_job = bqclient.query(sql_select)

print("\nQuery results:")
query_job.to_dataframe()


Query results:


,id,name,location,price_tier,checkin_date,checkout_date,booked
0,1,Hilton Basel,Basel,Luxury,2024-04-22,2024-04-20,False
1,6,InterContinental Geneva,Geneva,Luxury,2024-04-23,2024-04-28,False
2,4,Radisson Blu Lucerne,Lucerne,Midscale,2024-04-24,2024-04-05,False
3,10,Comfort Inn Bern,Bern,Midscale,2024-04-04,2024-04-16,False
4,5,Best Western Bern,Bern,Upper Midscale,2024-04-23,2024-04-01,False
5,8,Holiday Inn Basel,Basel,Upper Midscale,2024-04-24,2024-04-09,False
6,7,Sheraton Zurich,Zurich,Upper Upscale,2024-04-27,2024-04-02,False
7,2,Marriott Zurich,Zurich,Upscale,2024-04-14,2024-04-21,False
8,9,Courtyard Zurich,Zurich,Upscale,2024-04-03,2024-04-13,False
9,3,Hyatt Regency Basel,Basel,Upper Upscale,2024-04-10,2024-04-19,True


### Clean-Up
Conditionally delete BigQuery table and dataset in final session.

In [21]:
bqclient.delete_table(table_ref, not_found_ok=True)

bqclient.get_dataset(dataset_ref)
tables_in_dataset = list(bqclient.list_tables(dataset_ref))
if not tables_in_dataset:
    bqclient.delete_dataset(dataset_ref, delete_contents=False, not_found_ok=True)
    print(f"Dataset '{DATASET}' deleted.")
else:
    print(f"Dataset '{DATASET}' is not empty. Skipping dataset deletion.")

Dataset 'toolbox_ds' deleted.
